# Data Wrangling

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv")


## Finding duplicates

In [2]:
# Find how many duplicate rows exists in the df
duplicate_rows = df.duplicated().sum()

print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 154


## Removing duplicates

In [3]:
# Remove the duplicate rows from the dataframe
df_clean = df.drop_duplicates()

print(f"Number of rows after removing duplicates: {df_clean.shape[0]}")

Number of rows after removing duplicates: 11398


In [4]:
# Verifying if duplicates were dropped
duplicate_rows = df_clean.duplicated().sum()
duplicate_rows

0

## Finding Missing values

In [5]:
# Find the missing values for all columns
missing_values = df_clean.isnull().sum()

print(missing_values)

Respondent        0
MainBranch        0
Hobbyist          0
OpenSourcer       0
OpenSource       81
               ... 
Sexuality       542
Ethnicity       675
Dependents      140
SurveyLength     19
SurveyEase       14
Length: 85, dtype: int64


In [6]:
# Find out how many rows are missing in the column 'WorkLoc'
missing_workloc = df_clean['WorkLoc'].isnull().sum()

print(f"Number of missing rows in WorkLoc: {missing_workloc}")

Number of missing rows in WorkLoc: 32


## Imputing missing values


In [7]:
# Find the value counts for the column WorkLoc.

df_clean['WorkLoc'].value_counts() # counts freq of unique values in each column

WorkLoc
Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: count, dtype: int64

In [8]:
# Identify the value that is most frequent (majority) in the WorkLoc column.\
most_freq = df_clean['WorkLoc'].mode().iloc[0] # iloc[0] is used to get the first mode

print(most_freq)

Office


In [9]:
# Impute (replace) all the empty rows in the column WorkLoc with the value that you have identified as majority.

df_clean['WorkLoc'].replace('', most_freq, inplace=True) # replace empty strings
df_clean['WorkLoc'].fillna(most_freq, inplace=True) # replace NaN values

C:\Users\Steven\AppData\Local\Temp\ipykernel_172244\1716282950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['WorkLoc'].replace('', most_freq, inplace=True) # replace empty strings
C:\Users\Steven\AppData\Local\Temp\ipykernel_172244\1716282950.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['WorkLoc'].fillna(most_freq, inplace=True) # replace NaN values


In [10]:
# Veirfy if imputing was successful
update_missing = df_clean['WorkLoc'].isnull().sum()
print(update_missing)

0


# Normalizing data

There are two columns in the dataset that talk about compensation.

One is "CompFreq". This column shows how often a developer is paid (Yearly, Monthly, Weekly).

The other is "CompTotal". This column talks about how much the developer is paid per Year, Month, or Week depending upon his/her "CompFreq".

This makes it difficult to compare the total compensation of the developers.

Create a new column called 'NormalizedAnnualCompensation' which contains the 'Annual Compensation' irrespective of the 'CompFreq'.

It makes comparison of salaries easy.


### List out the various categories in the column 'CompFreq'

In [11]:
categories = df_clean['CompFreq'].value_counts()
print(categories)

CompFreq
Yearly     6073
Monthly    4788
Weekly      331
Name: count, dtype: int64


### Create a new column named 'NormalizedAnnualCompensation'. 

In [12]:

def calculate_Annual_Compensation(row):
    if row['CompFreq'] == 'Yearly':
        return row['CompTotal']
    elif row['CompFreq'] == 'Monthly':
        return row['CompTotal'] * 12
    elif row['CompFreq'] == 'Weekly':
        return row['CompTotal'] * 52
    else:
        return np.nan
    
df_clean['NormalizedAnnualCompensation'] = df_clean.apply(calculate_Annual_Compensation, axis=1)

C:\Users\Steven\AppData\Local\Temp\ipykernel_172244\12108731.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['NormalizedAnnualCompensation'] = df_clean.apply(calculate_Annual_Compensation, axis=1)


In [16]:
compensation = df_clean['NormalizedAnnualCompensation']
compensation

0         61000.0
1        138000.0
2         90000.0
3        348000.0
4         90000.0
           ...   
11547    130000.0
11548     74400.0
11549    105000.0
11550     80000.0
11551         NaN
Name: NormalizedAnnualCompensation, Length: 11398, dtype: float64